In [2]:
import requests, pandas as pd
from itertools import product
from tabulate import tabulate

In [ ]:
dataset = 'sts_inpp_m' 
url = f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{dataset}?geo=IT&format=JSON"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # print("- ✅ Connessione riuscita.")
    print("- 📦 Chiavi:", data.keys(), "\n")
    print("- 📐 Dimensioni disponibili:", data["dimension"].keys(), "\n")
    print("- 🔢 Numero di valori:", len(data["value"]), "\n")
else:
    print("❌ Errore:", response.status_code)
    print(response.text)

❌ Errore: 404
{ "error": [{"status": 404,"id": 100,"label": "ERR_NOT_FOUND_4: EI_PRIC_IM_M (DATA_FLOW:ALL,1.0) is not available for dissemination."}]}


In [6]:
from itertools import product
import pandas as pd

def parse_eurostat_generic(data):
    """
    Estrae e organizza i dati Eurostat (es. sts_inpp_m) in un DataFrame.

    Parametri:
    - data: dict, risposta JSON da Eurostat API

    Ritorna:
    - pd.DataFrame con una colonna per ciascuna dimensione + 'value'
    """
    dims = data["dimension"]
    dim_order = data["id"]      # Ordine dimensioni
    dim_sizes = data["size"]    # Numero di elementi per dimensione

    # Etichette per ogni dimensione
    labels = {dim: dims[dim]["category"]["label"] for dim in dim_order}
    keys = {dim: list(labels[dim].keys()) for dim in dim_order}

    # Prodotto cartesiano di tutte le combinazioni
    combinations = list(product(*[keys[dim] for dim in dim_order]))

    # Ricostruzione DataFrame
    records = []
    for i, combo in enumerate(combinations):
        if str(i) in data["value"]:
            record = {}
            for dim, code in zip(dim_order, combo):
                record[f"{dim}_code"] = code
                record[f"{dim}_label"] = labels[dim][code]
            record["value"] = data["value"][str(i)]
            records.append(record)

    return pd.DataFrame(records)


In [7]:
df = parse_eurostat_generic(data)
df.head()


,freq_code,freq_label,indic_bt_code,indic_bt_label,nace_r2_code,nace_r2_label,s_adj_code,s_adj_label,unit_code,unit_label,geo_code,geo_label,time_code,time_label,value
0,M,Monthly,PRC_PRR,Producer prices,C_HTC,High-technology manufacturing,NSA,Unadjusted data (i.e. neither seasonally adjus...,I21,"Index, 2021=100",IT,Italy,2000-01,2000-01,101.4
1,M,Monthly,PRC_PRR,Producer prices,C_HTC,High-technology manufacturing,NSA,Unadjusted data (i.e. neither seasonally adjus...,I21,"Index, 2021=100",IT,Italy,2000-02,2000-02,101.5
2,M,Monthly,PRC_PRR,Producer prices,C_HTC,High-technology manufacturing,NSA,Unadjusted data (i.e. neither seasonally adjus...,I21,"Index, 2021=100",IT,Italy,2000-03,2000-03,101.5
3,M,Monthly,PRC_PRR,Producer prices,C_HTC,High-technology manufacturing,NSA,Unadjusted data (i.e. neither seasonally adjus...,I21,"Index, 2021=100",IT,Italy,2000-04,2000-04,101.6
4,M,Monthly,PRC_PRR,Producer prices,C_HTC,High-technology manufacturing,NSA,Unadjusted data (i.e. neither seasonally adjus...,I21,"Index, 2021=100",IT,Italy,2000-05,2000-05,101.7


In [8]:
df['dataset'] = dataset
print(df.head())

import sqlite3
conn = sqlite3.connect('D:/files/Eurostat.sqlite')  # o altro percorso
df.to_sql(f"ITA_{dataset}", conn, if_exists='replace', index=False)
print(f"✅ Tabella 'ITA_{dataset}' salvata con {len(df)} righe.")
conn.close()

  freq_code freq_label indic_bt_code   indic_bt_label nace_r2_code  \
0         M    Monthly       PRC_PRR  Producer prices        C_HTC   
1         M    Monthly       PRC_PRR  Producer prices        C_HTC   
2         M    Monthly       PRC_PRR  Producer prices        C_HTC   
3         M    Monthly       PRC_PRR  Producer prices        C_HTC   
4         M    Monthly       PRC_PRR  Producer prices        C_HTC   

                   nace_r2_label s_adj_code  \
0  High-technology manufacturing        NSA   
1  High-technology manufacturing        NSA   
2  High-technology manufacturing        NSA   
3  High-technology manufacturing        NSA   
4  High-technology manufacturing        NSA   

                                         s_adj_label unit_code  \
0  Unadjusted data (i.e. neither seasonally adjus...       I21   
1  Unadjusted data (i.e. neither seasonally adjus...       I21   
2  Unadjusted data (i.e. neither seasonally adjus...       I21   
3  Unadjusted data (i.e. neither